In [ ]:
import moth
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import scipy
import scipy.signal
import h5py
import pathlib

PATH = '/storage/home/adz6/group/project'
DIGDATAPATH = os.path.join(PATH, 'datasets', 'data', 'lab', 'dig')
#DAQDATAPATH = os.path.join(PATH, 'daq_data')
#MOTHPATH = os.path.join(PATH, 'moth')
#MOTHDATA = os.path.join(MOTHPATH, 'data/datasets')
#PLOTPATH = os.path.join(PATH, 'plots/moth')
#MOTHRESULTS = os.path.join(PATH, 'results/moth')
sns.set_theme(context='talk')

def SumData(array_data, xpos, ypos, nch, idealx, r=0.1, f=25.8e9, antispiral=False):
    
    xx, yy = np.meshgrid(xpos, ypos)
    array_antenna_theta = np.arange(0, 2 * np.pi, 2 * np.pi / array_data.shape[0])
    
    xantenna = (r * np.cos(-1 * array_antenna_theta)).reshape((array_antenna_theta.size, 1, 1))
    yantenna = (r * np.sin(-1 * array_antenna_theta)).reshape((array_antenna_theta.size, 1, 1))
    
    xdist = xantenna - xx.reshape((1, *xx.shape))
    ydist = yantenna - yy.reshape((1, *yy.shape))
    
    dist_grid = np.sqrt(xdist ** 2 + ydist ** 2)
    
    ind_max = np.argmax(abs(np.fft.fft(array_data[:, nch, :], axis=-1)), axis=-1)[0]
    
    #print(np.angle(np.fft.fft(array_data[:, nch, :], axis=-1))[:, ind_max])
    
    zero = np.angle(np.fft.fft(array_data[:, nch, :], axis=-1))[0, ind_max]
    
    ideal_antispiral = np.linspace(0, 2 * np.pi, 60)
    ideal_x = idealx
    ideal_y = 0.0
    
    ideal_distances = np.sqrt(
        (r * np.cos(-1 * array_antenna_theta) - ideal_x) ** 2
        + (r * np.sin(-1 * array_antenna_theta) - ideal_y) ** 2
    )
    
    ideal_phases =  np.unwrap(np.roll(2 * np.pi * ideal_distances * f / 3e8 - ideal_antispiral, -15)) 
    #ideal_phases -= (zero + ideal_phases[0])
    
    array_phases = np.unwrap(np.angle(np.fft.fft(array_data[:, nch, :], axis=-1))[:, ind_max])
    ideal_phases -= (ideal_phases[0] - array_phases[0])
    ideal_phases += np.pi/4
    
    diff = ideal_phases - array_phases
    
    fig = plt.figure(figsize=(13, 8))
    ax = fig.add_subplot(1,1,1)
    
    #ax.plot(np.angle(np.fft.fft(array_data[:, nch, :], axis=-1))[:, ind_max])
    ax.plot(ideal_phases)
    plt.show()
    
    fig = plt.figure(figsize=(13, 8))
    ax = fig.add_subplot(1,1,1)
    
    ax.plot(array_phases)
    ax.plot(ideal_phases)
    ax.plot(diff)
    #ax.plot(np.unwrap(np.angle(np.fft.fft(array_data[:, nch, :], axis=-1))[:, ind_max] - np.roll(diff, 15)))
    
    plt.show()
    
    #xx, yy = np.expand_dims(xx, axis=0).repeat(array_data['angle'].size, axis=0), np.expand_dims(yy, axis=0).repeat(array_data['angle'].size, axis=0)

    
    amplitude_weight = 1 #/ dist_grid
    #amplitude_weight = amplitude_weight * (array_antenna_theta.size / amplitude_weight.sum(axis=0)).reshape((1, amplitude_weight.shape[1], amplitude_weight.shape[2]))

    beamforming_phases = np.exp(-1j * 2 * np.pi * dist_grid * f / 3e8)
    
    if antispiral:
        
        #grid_angles = np.arctan2(ydist, xdist)
        
        #antispiral_phase = np.exp(-1j * grid_angles)
        phase_shift = np.exp(-1j *  
                                  (-1*array_antenna_theta -0 * diff)
                                 ).reshape((60, 1, 1))
        #antispiral_phase = np.exp(-1j * 
        #                          (array_antenna_theta + diff)
        #                         ).reshape((60, 1, 1))
        
        
        return np.matmul((amplitude_weight * beamforming_phases * phase_shift).T, array_data[:, nch, :])
    else:
        phase_shift = np.exp(-1j *  
                                  (-0*array_antenna_theta -1 * diff)
                                 ).reshape((60, 1, 1))
        return np.matmul((amplitude_weight * beamforming_phases * phase_shift).T, array_data[:, nch, :])



In [ ]:
os.listdir(DIGDATAPATH)

In [ ]:
file = h5py.File(os.path.join(DIGDATAPATH, '211213_cres2_meas_3ch.h5'))

In [ ]:
file['range3cm_step1mm_z0mm'].keys()

In [ ]:
file['range3cm_step1mm_z0mm']['10'].shape

# combine signals

In [ ]:
data_array = np.zeros((60, 32758), np.complex128)

select_antennas = np.arange(0, 120, 2)
# channel 0-30 from antenna 1
# channel 30-60 from antenna 3

data_array[0:30, :] = file['range3cm_step1mm_z0mm']['10'][select_antennas[0:30], 1, :]
data_array[30:, :] = file['range3cm_step1mm_z0mm']['10'][select_antennas[0:30], 3, :]

data_array = data_array.reshape((data_array.shape[0], 1, data_array.shape[-1]))
print(data_array.shape)



In [ ]:
test_data = file['range3cm_step1mm_z0mm']['10'][:]

In [ ]:
idealx = 0.01

x = np.linspace(-0.05, 0.05, 101)
y = np.linspace(-0.05, 0.05, 101)

summed_data_no_antispiral = SumData(test_data[select_antennas, :, :], x, y, 1, idealx, antispiral=False, )
summed_data = SumData(test_data[select_antennas, :, :], x, y, 1, idealx, antispiral=True, )

In [ ]:
image = np.mean(abs(summed_data) ** 2, axis = -1)
image_no_spiral = np.mean(abs(summed_data_no_antispiral) ** 2, axis=-1)

In [ ]:
sns.set_theme(style='ticks', context='talk', font_scale=1.3)
cmap = sns.color_palette('plasma', as_cmap=True)

fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(1,1,1)

norm = np.max(image)

image_norm = image / norm
image_no_spiral_norm = image_no_spiral / norm

img = ax.imshow(
    image_no_spiral_norm,
    aspect='auto',
    cmap=cmap,
    extent=(-50, 50, 50, -50),
    vmin=0,
    vmax=1
)
cbar = fig.colorbar(img, label=r'Power (AU)')
ax.set_xlabel('X (mm)')
ax.set_ylabel('Y (mm)')

plt.tight_layout()
save_path = pathlib.Path.home()/'group'/'project'/'plots'/'analysis'/'antenna'/'cyclotron_emitter'
name = '220729_10mm_beamforming_no_antispiral'

plt.savefig(save_path/name)